In [1]:
import torch
import torch.nn as nn

In [4]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)

  def forward(self, x):
    return self.linear(x)


if __name__ == '__main__':
  x_data = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
  y_data = [2, 2, 2, 1, 1, 1, 0, 0]     # 답은 0, 1, 2 이렇게 3개만 존재한다고 가정
  x_train = torch.FloatTensor(x_data)
  y_train = torch.LongTensor(y_data)

  model = SoftmaxClassifierModel()
  criterion = nn.CrossEntropyLoss()    # 이미 nn.LogSoftmax가 포함되어 있다.
  optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

  nb_epochs = 3000

  for epoch in range(nb_epochs+1):
    prediction = model(x_train)

    loss = criterion(prediction, y_train)    # criterion(input, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
      print('Epoch {:4d}/{} Loss: {:.6f}'.format(
          epoch, nb_epochs, loss.item()
      ))

Epoch    0/3000 Loss: 2.113073
Epoch  100/3000 Loss: 0.648366
Epoch  200/3000 Loss: 0.560265
Epoch  300/3000 Loss: 0.504935
Epoch  400/3000 Loss: 0.460156
Epoch  500/3000 Loss: 0.420397
Epoch  600/3000 Loss: 0.383207
Epoch  700/3000 Loss: 0.347037
Epoch  800/3000 Loss: 0.310732
Epoch  900/3000 Loss: 0.274023
Epoch 1000/3000 Loss: 0.244529
Epoch 1100/3000 Loss: 0.231241
Epoch 1200/3000 Loss: 0.220042
Epoch 1300/3000 Loss: 0.209847
Epoch 1400/3000 Loss: 0.200524
Epoch 1500/3000 Loss: 0.191967
Epoch 1600/3000 Loss: 0.184084
Epoch 1700/3000 Loss: 0.176801
Epoch 1800/3000 Loss: 0.170052
Epoch 1900/3000 Loss: 0.163781
Epoch 2000/3000 Loss: 0.157940
Epoch 2100/3000 Loss: 0.152487
Epoch 2200/3000 Loss: 0.147386
Epoch 2300/3000 Loss: 0.142604
Epoch 2400/3000 Loss: 0.138113
Epoch 2500/3000 Loss: 0.133888
Epoch 2600/3000 Loss: 0.129905
Epoch 2700/3000 Loss: 0.126146
Epoch 2800/3000 Loss: 0.122592
Epoch 2900/3000 Loss: 0.119227
Epoch 3000/3000 Loss: 0.116037


###**MNIST**

In [10]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import time

In [7]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다: ", device)

다음 기기로 학습합니다:  cpu


In [8]:
random.seed(777)
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [26]:
# hyperparameter
training_epochs = 10
batch_size = 100


# MNIST dataset
train_dataset = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

test_dataset = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)


# dataset loader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True      # batch size에 data 개수가 딱 떨어지지 않으면 마지막 batch는 데이터 숫자가 적게 된다.
                          )                   # 이때 그냥 적은 데이터 숫자로 학습을 진행하면 마지막 batch가 과대평가 될 수 있다.
                                               # drop_last는 True를 할당하면 뒤의 남는 data를 그냥 버려준다.
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True
                         )


class MNIST_classification(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(784, 10, bias=True)   

  def forward(self, x):
    return self.linear(x.view(-1, 784))


model = MNIST_classification()
model.to(device)   # 뒤의 to(device)는 gpu를 사용할 때 필요하다.

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)


def train(epoch):
  model.train()    #train을 위한 형식
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()     #학습을 완료할 때마다 gradient들을 0으로 초기화 해줘야 한다.
    output = model(data)      #output 계산
    loss = criterion(output, target)     #loss 계산
    loss.backward()     #편미분 값 계산
    optimizer.step()     #weight update

    if batch_idx % 10 == 0:
      print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
          epoch, batch_idx*len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))



def test():
  model.eval()    #test를 위한 형식
  test_loss = 0
  correct = 0

  for data, target in test_loader:      #답을 모를땐 target 부분이 존재하지 않는다. MNIST의 경우 답이 존재하므로 답 체크를 위해 써놓는 것
    data, target = data.to(device), target.to(device)
    output = model(data)

    # sum up batch loss
    test_loss += criterion(output, target).item()     # loss.item()은 loss 스칼라 값을 준다.

    # get the index of the max
    # output.data는 100 x 10 즉 100개의 data들에 대한 0~9까지에 대한 확률 값인 1x10이고
    # max(1, keepdim=True) 첫번째 인자가 1인 것은 0~9 중 무엇이 가장 큰지 찾으라는 것이고 
    # 두번째 인자는 차원 유지 즉 100 x 1로 유지해준다. False로 하면 그냥 100됨
    # max는 (value, index)를 반환하는데 여기다가 [1]을 했으므로 index를 찾게 된다. 즉 예측 label을 찾은 것이다.
    pred = output.data.max(1, keepdim=True)[1]      

    # .eq()를 통하여 예측값과 traget이 같은 지 계산해줍니다.
    # target.data.view_as(pred) 모양을 바꿔 target을 pred랑 똑같은 차원이 되도록 한다.
    # 한 batch 당 100개의 image가 나오는데 100 x 1의 tensor에다가 예측 label이 target과 같으면 1 아니면 0을 부여하므로 sum()을 하면 맞춘 횟수가 된다.
    correct +=  pred.eq(target.data.view_as(pred)).cpu().sum()    

    # 즉 correct는 총 맞춘 횟수이다.

  test_loss /= len(test_loader.dataset)
  print(f'==================\nTest set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
        f'({100. * correct / len(test_loader.dataset):.0f}%)')
 


## Main

if __name__ == '__main__':
 
  since = time.time()
  for epoch in range(1, training_epochs + 1):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Tesing time: {m:.0f}m {s:.0f}s')

  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/60000 (0%) | Loss : 2.330015
Train Epoch : 1 | Batch Status : 1000/60000 (2%) | Loss : 1.595296
Train Epoch : 1 | Batch Status : 2000/60000 (3%) | Loss : 1.253153
Train Epoch : 1 | Batch Status : 3000/60000 (5%) | Loss : 1.005576
Train Epoch : 1 | Batch Status : 4000/60000 (7%) | Loss : 0.837487
Train Epoch : 1 | Batch Status : 5000/60000 (8%) | Loss : 0.883809
Train Epoch : 1 | Batch Status : 6000/60000 (10%) | Loss : 0.758899
Train Epoch : 1 | Batch Status : 7000/60000 (12%) | Loss : 0.681512
Train Epoch : 1 | Batch Status : 8000/60000 (13%) | Loss : 0.644894
Train Epoch : 1 | Batch Status : 9000/60000 (15%) | Loss : 0.683423
Train Epoch : 1 | Batch Status : 10000/60000 (17%) | Loss : 0.571650
Train Epoch : 1 | Batch Status : 11000/60000 (18%) | Loss : 0.635151
Train Epoch : 1 | Batch Status : 12000/60000 (20%) | Loss : 0.563257
Train Epoch : 1 | Batch Status : 13000/60000 (22%) | Loss : 0.650273
Train Epoch : 1 | Batch Status : 14000/60000 (23%) | 